In [1]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import time
import json
import os
import pandas as pd

In [2]:
# Selenium modules
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# Run Google Chrome from Python.
chrome_path = 'C:\WebDrivers\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)

## NASA Mars News

In [4]:
# Open the webpage from Python.
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
driver.get(url)

time.sleep(10)

In [5]:
# Get the HTML for the website.
html = driver.execute_script('return document.documentElement.outerHTML')

#Close the driver
driver.close()

#html

In [6]:
# This is the HTML for the webpage
soup = BeautifulSoup(html, 'lxml')
#print(soup.prettify())

In [7]:
page = soup.find('div', id = 'page')

# Find all <li> tags
all_lis = [li for li in page.ul.find_all('li')]
len(all_lis)

40

In [8]:
# Create mars_dict to collect the information along the way:
mars_dict = {}

In [9]:
# The latest news is in the first <li> tag.
latest_li = all_lis[0]

latest_news_title = latest_li.find('div', class_ = 'content_title').text
latest_news_p = latest_li.find('div', class_ = 'article_teaser_body').text
print(latest_news_title)
print(latest_news_p)

NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet
The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. 


In [10]:
mars_dict["latest_news_title"] = latest_news_title
mars_dict["latest_news_p"] = latest_news_p

## JPL Mars Space Images - Featured Image

In [11]:
# Run Google Chrome from Python.
url_JPL_Mars = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
driver = webdriver.Chrome(chrome_path)
driver.get(url_JPL_Mars)

In [12]:
# Get the HTML of the current page you are on.
image_html = driver.page_source

#html

In [13]:
# This is the HTML for the webpage
image_soup = BeautifulSoup(image_html, 'lxml')
#print(image_soup.prettify())

In [14]:
# Search for the tag of the full image, using link text.
image_link = driver.find_element_by_link_text('FULL IMAGE')

# Click on the link for the full image.
image_link.click()

# Wait for conpletion of previous action.
try:
    element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.LINK_TEXT, "more info"))
    )
    
    # Click on the button for more info.
    element.click()
except:
    driver.quit()

In [15]:
# Get the HTML of the current page for the large picture.
large_html = driver.page_source
#large_html

#Close the driver
driver.close()

# This is the HTML for the full picture webpage
large_soup = BeautifulSoup(large_html, 'lxml')
#print(large_soup.prettify())

In [16]:
figure_tag = large_soup.find('figure', class_ = 'lede')
base_url = 'https://www.jpl.nasa.gov'
featured_image_url = base_url+figure_tag.a['href']
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18816_hires.jpg


In [17]:
mars_dict["featured_image_url"] = featured_image_url

## Mars Weather

In [18]:
# Run Google Chrome from Python.
url_weather = 'https://twitter.com/marswxreport?lang=en'
driver = webdriver.Chrome(chrome_path)
driver.get(url_weather)
time.sleep(10)

In [19]:
# Get the HTML of the current page you are on.
weather_html = driver.page_source

#weather_html

#Close the driver
driver.close()

In [20]:
# This is the HTML for the Twitter webpage on Mars weather.
weather_soup = BeautifulSoup(weather_html, 'lxml')

#print(weather_soup.prettify())

In [21]:
div_tag = weather_soup.find('div', class_ = 'css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0')
mars_weather = div_tag.text
print(mars_weather)

InSight sol 598 (2020-08-01) low -91.6ºC (-132.9ºF) high -15.1ºC (4.8ºF)
winds from the WNW at 7.1 m/s (15.8 mph) gusting to 19.2 m/s (43.0 mph)
pressure at 7.90 hPa


In [22]:
mars_dict["mars_weather"] = mars_weather

## Mars Facts

In [23]:
# Fetch the page at the url using "requests" module.
facts_url = 'https://space-facts.com/mars/'
response = requests.get(facts_url)

In [24]:
# Read all tables in the response into a list of dataframes
dfs=pd.read_html(response.text)

In [25]:
# Iterate through the DataFrames to access each table.
facts_df = dfs[0]
facts_df.columns = ['variable', 'value']
facts = facts_df.set_index('variable')
facts

,value
variable,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [26]:
# Save DataFrame as HTML.
facts_html_path = os.path.join(".", "static", "facts.html")
facts.to_html(facts_html_path, encoding="utf-8", index=True)

In [27]:
mars_df = facts.T
mars_df.columns = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons',
       'Orbit Distance', 'Orbit Period', 'Surface Temperature',
       'First Record', 'Recorded By']
mars_df

,Equatorial Diameter,Polar Diameter,Mass,Moons,Orbit Distance,Orbit Period,Surface Temperature,First Record,Recorded By
value,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 (Phobos & Deimos),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [28]:
# Save DataFrame as HTML.
mars_html_path = os.path.join(".", "static", "mars_facts.html")
mars_df.to_html(mars_html_path, encoding="utf-8", index=False)

## Mars Hemispheres

In [29]:
# Run Google Chrome from Python.
url_hemis = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
driver = webdriver.Chrome(chrome_path)
driver.get(url_hemis)

hemisphere_image_urls = []

## Cerberus Hemisphere

In [30]:
# Search for the tag of the full image, using link text.
cerberus_link = driver.find_element_by_link_text('Cerberus Hemisphere Enhanced')

# Click on the link for the full image.
cerberus_link.click()

# Get the HTML of the current page.
cerberus_hemis_html = driver.page_source
#cerberus_hemis_html

# This is the HTML for the full picture webpage
cerberus_soup = BeautifulSoup(cerberus_hemis_html, 'lxml')
#print(cerberus_hemis_soup.prettify())

cerberus_title = cerberus_soup.find('h2', class_ = 'title').text[:-9]
print(cerberus_title)

mars_dict["cerberus_title"] = cerberus_title

cerberus_div_img = cerberus_soup.find('div', class_ = 'downloads')
cerberus_img_url = cerberus_div_img.a['href']
print(cerberus_img_url)

mars_dict["cerberus_img_url"] = cerberus_img_url

# Dictionary
cerberus_dict = {}
cerberus_dict['title'] = cerberus_title
cerberus_dict['img_url'] = cerberus_img_url

hemisphere_image_urls.append(cerberus_dict)

driver.back()

Cerberus Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg


## Schiaparelli Hemisphere

In [31]:
# Search for the tag.
schiaparelli_link = driver.find_element_by_link_text('Schiaparelli Hemisphere Enhanced')

# Click on the link.
schiaparelli_link.click()

# Get the HTML of the current page.
schiaparelli_html = driver.page_source
#schiaparelli_html

# This is the HTML for the current webpage
schiaparelli_soup = BeautifulSoup(schiaparelli_html, 'lxml')
#print(schiaparelli_soup.prettify())

schiaparelli_title = schiaparelli_soup.find('h2', class_ = 'title').text[:-9]
print(schiaparelli_title)

mars_dict["schiaparelli_title"] = schiaparelli_title

schiaparelli_div_img = schiaparelli_soup.find('div', class_ = 'downloads')
schiaparelli_img_url = schiaparelli_div_img.a['href']
print(schiaparelli_img_url)

mars_dict["schiaparelli_img_url"] = schiaparelli_img_url

# Dictionary
schiaparelli_dict = {}
schiaparelli_dict['title'] = schiaparelli_title
schiaparelli_dict['img_url'] = schiaparelli_img_url

hemisphere_image_urls.append(schiaparelli_dict)

driver.back()

Schiaparelli Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg


## Syrtis Major Hemisphere

In [32]:
# Search for the tag.
syrtis_link = driver.find_element_by_link_text('Syrtis Major Hemisphere Enhanced')

# Click on the link.
syrtis_link.click()

# Get the HTML of the current page.
syrtis_html = driver.page_source
#syrtis_html

# This is the HTML for the current webpage
syrtis_soup = BeautifulSoup(syrtis_html, 'lxml')
#print(syrtis_soup.prettify())

syrtis_title = syrtis_soup.find('h2', class_ = 'title').text[:-9]
print(syrtis_title)

mars_dict["syrtis_title"] = syrtis_title

syrtis_div_img = syrtis_soup.find('div', class_ = 'downloads')
syrtis_img_url = syrtis_div_img.a['href']
print(syrtis_img_url)

mars_dict["syrtis_img_url"] = syrtis_img_url

# Dictionary
syrtis_dict = {}
syrtis_dict['title'] = syrtis_title
syrtis_dict['img_url'] = syrtis_img_url

hemisphere_image_urls.append(syrtis_dict)

driver.back()

Syrtis Major Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg


## Valles Marineris Hemisphere Enhanced

In [33]:
# Search for the tag.
valles_link = driver.find_element_by_link_text('Valles Marineris Hemisphere Enhanced')

# Click on the link.
valles_link.click()

# Get the HTML of the current page.
valles_html = driver.page_source
#syrtis_html

# This is the HTML for the current webpage
valles_soup = BeautifulSoup(valles_html, 'lxml')
#print(valles_soup.prettify())

valles_title = valles_soup.find('h2', class_ = 'title').text[:-9]
print(valles_title)

mars_dict["valles_title"] = valles_title

valles_div_img = valles_soup.find('div', class_ = 'downloads')
valles_img_url = valles_div_img.a['href']
print(valles_img_url)

mars_dict["valles_img_url"] = valles_img_url

# Dictionary
valles_dict = {}
valles_dict['title'] = valles_title
valles_dict['img_url'] = valles_img_url

hemisphere_image_urls.append(valles_dict)

driver.back()

#Close the driver
driver.close()

Valles Marineris Hemisphere
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [34]:
# Print mars_dict
for key, value in mars_dict.items():
    print(f'{key}:   {value}')

latest_news_title:   NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet
latest_news_p:   The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. 
featured_image_url:   https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18816_hires.jpg
mars_weather:   InSight sol 598 (2020-08-01) low -91.6ºC (-132.9ºF) high -15.1ºC (4.8ºF)
winds from the WNW at 7.1 m/s (15.8 mph) gusting to 19.2 m/s (43.0 mph)
pressure at 7.90 hPa
cerberus_title:   Cerberus Hemisphere
cerberus_img_url:   https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
schiaparelli_title:   Schiaparelli Hemisphere
schiaparelli_img_url:   https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
syrtis_title:   Syrtis Major Hemisphere
syrtis_img_url:   https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
valles_title:   Valles Mar